In [1]:
%load_ext autoreload
%autoreload 2

from warnings import filterwarnings

import torch
import torch.nn as nn

import config_iProL as config
from dataset import get_datasets, get_cv_dataloader_list
from dataset import get_excel, get_complete_data
from module import Longformer_base_Net
from tools import check_result_dir
from tools_fit import model_fit

『PBC note』Longformer_base_Net相比base，没有LSTM


c:\Users\Hieu\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Hieu\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Users\Hieu\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime r

d:\Project\DNA\iProL\src\feature_extraction\__init__.py


In [2]:
sheet = get_excel(version='', sheet_name='dataset')
complete_data = get_complete_data(sheet)

dataset_identify, _, _ = get_datasets(complete_data)

identify_dataset_list, identify_loader_list = get_cv_dataloader_list(config, dataset_identify, "Pro")

d:\Project\DNA\iProL\src\dataset.py:81: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataset_level["Con_level"] = dataset_level["Con_level"].replace(["Strong", "Weak"], [1, 0])
d:\Project\DNA\iProL\src\dataset.py:85: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataset_type["Sigma"] = dataset_type["Sigma"].replace(sigma_list, label_list)


PBC[ProL] --- kmer = 2
train len: 3382 test len: 3382
PBC[ProL] --- kmer = 2
train len: 3382 test len: 3382


In [3]:
filterwarnings("ignore")
# torch.manual_seed(0)
# torch.cuda.manual_seed(0)

In [4]:
"""
识别promoter：Pro
"""
for cv_idx, (train_loader, test_loader) in enumerate(identify_loader_list, 1):
    check_result_dir(config, cv_idx)
    # model = Longformer_base_lstm_Net(config.model_name, config.device,
    #                                  config.add_special_tokens,
    #                                  bidirectional=config.bidirectional,
    #                                  num_layers=config.num_layers)
    """去掉一个CNN，dense1参数 in_channels进入"""
    # model = Longformer_base_2cnn_lstm_Net(config.model_name, config.device,
    #                                  config.add_special_tokens,
    #                                  bidirectional=config.bidirectional,
    #                                  num_layers=config.num_layers)
    """Longformer_base_lstm_BNpos_Net: 修改BN到pooling之后dropout之前， 美纱用"""
    # model = Longformer_base_lstm_BNpos_Net(config.model_name, config.device,
    #                                  config.add_special_tokens,
    #                                  bidirectional=config.bidirectional,
    #                                  num_layers=config.num_layers)
    """
    没有LSTM
    """
    model = Longformer_base_Net(config.model_name, config.device,
                                     config.add_special_tokens)
    model.to(config.device)

    # 这里是一般情况，共享层往往不止一层，所以做一个for循环
    if hasattr(model, 'longformer') and config.not_fine_tuning:
        for para in model.longformer.parameters():
            para.requires_grad = False
        # print(module.parameters())

    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=config.lr)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 50, gamma=0.6)
    criterion = nn.BCELoss()

    model_fit(config, model, train_loader, test_loader, optimizer, criterion, scheduler, 0.86, cv_idx)


Ex result dir d:\Project\DNA\iProL\src\../result/longformer/_train[kmer2_bs64_lr0.0005_epoch25]_pm[astFalse_nftTrue_levelbase]\ProL_1216232630_Pro\1 has been created!
>>>>>>>>>>『CV START』[2/1-->lr:[0.0005]]>>>>>>>>>>>>
>>>>>>>>>>『CV START』[2/1]>>>>>>>>>>>>
『Computer device』nvidia:ProL
『dataset』kmer: 2, cv: 2
『train config』epochs: 25, lr: 0.0005, batch_size: 64, device: cuda
『pre-model config [True]』add_special_tokens: False, not_fine_tuning: True, model_name: longformer-base-4096
『ex result config』ex_result_dir: d:\Project\DNA\iProL\src\../result/longformer/_train[kmer2_bs64_lr0.0005_epoch25]_pm[astFalse_nftTrue_levelbase]\ProL_1216232630_Pro


Input ids are automatically padded to be a multiple of `config.attention_window`: 512


epoch 1 [53/53] total_loss = 0.607885608695588 lr = 0.0005, total=3382, TP=1197, TN=1160, FP=531, FN=494; precision=0.693, recall=0.708, Sn=0.708, Sp=0.686
roc_auc: (test=0.764) average_precision: (test=0.758) accuracy: (test=0.697) matthews_corrcoef: (test=0.394) f1: (test=0.700) 
epoch 1 [53/53] total_loss = 0.5985800736355331 total=3382, TP=1532, TN=926, FP=765, FN=159; precision=0.667, recall=0.906, Sn=0.906, Sp=0.548
roc_auc: (test=0.851) average_precision: (test=0.852) accuracy: (test=0.727) matthews_corrcoef: (test=0.486) f1: (test=0.768) 

epoch 2 [53/53] total_loss = 0.505197323155853 lr = 0.0005, total=3382, TP=1263, TN=1326, FP=365, FN=428; precision=0.776, recall=0.747, Sn=0.747, Sp=0.784
roc_auc: (test=0.833) average_precision: (test=0.834) accuracy: (test=0.766) matthews_corrcoef: (test=0.531) f1: (test=0.761) 
epoch 2 [53/53] total_loss = 0.5339869733126659 total=3382, TP=1191, TN=1473, FP=218, FN=500; precision=0.845, recall=0.704, Sn=0.704, Sp=0.871
roc_auc: (test=0.86

KeyboardInterrupt: 

In [5]:
import time

now_time = time.strftime('%m%d%H%M%S', time.localtime())
print(f"『now_time』:{now_time}")

『now_time』:1216230130


In [5]:
# clear memory
import gc
gc.collect()
torch.cuda.empty_cache()